<h1>Importing necessary packages</h1>

In [2]:
import numpy as np
import pandas as pd

<h1>Reading the dataset</h1>

In [3]:
df = open('msnbc990928.txt', "r")
lines = df.readlines()

for index, line in enumerate(lines):
      lines[index] = list(map(int, line.strip().split()))
R = sum(1 for line in lines)
C = 17
matrix = np.zeros((R, C), dtype=int)
R

989818

<h1>Creating the matrix and converting it to a DataFrame</h1>

In [4]:
for index, line in enumerate(lines):
    for item in line:
        matrix[index, item-1]=1

In [5]:
df = pd.DataFrame(data=matrix)
df.columns = pd.RangeIndex(1, len(df.columns)+1) 
df.index = pd.RangeIndex(1, len(df.index)+1) 
df = df.astype('bool')

<h1>Function for creating C1 itemset</h1>

In [6]:
C1=[]
def create_c1(data_set):
    for item in data_set.columns.values:
        if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))

<h1>Functrin for creating Lk</h1>

In [7]:
def generate_Lk_by_Ck(data_set, Ck, min_support, support_data):
    Lk = set()
    item_count = {}
    for t in Ck:
        item_count[t] = 0
        for item in data_set[t].values.tolist():
            if all(item) == True:
                item_count[t] += 1
    t_num = float(len(data_set))
    for item in item_count:
        if (item_count[item] / t_num) >= min_support:
            Lk.add(item)
            support_data[item] = item_count[item] / t_num
    return Lk

<h1>Check if the aggregated item is in L1 itemset</h1>

In [ ]:
def is_apriori(Ck_item, Lksub1):
    for item in Ck_item:
        sub_Ck = Ck_item - frozenset([item])
        if sub_Ck not in Lksub1:
            return False
    return True

<h1>Function for creating Ck</h1>

In [8]:
def create_Ck(Lksub1, k):
    Ck = set()
    len_Lksub1 = len(Lksub1)
    list_Lksub1 = list(Lksub1)
    for i in range(len_Lksub1):
        for j in range(1, len_Lksub1):
            l1 = list(list_Lksub1[i])
            l2 = list(list_Lksub1[j])
            l1.sort()
            l2.sort()
            if l1[0:k-2] == l2[0:k-2]:
                Ck_item = list_Lksub1[i] | list_Lksub1[j]
                # pruning
                if is_apriori(Ck_item, Lksub1):
                    Ck.add(Ck_item)
    return Ck

<h1>Generating Lk</h1>

In [10]:
def generate_L(data_set, k, min_support):
    support_data = {}
    c1 = create_c1(data_set)
    L1 = generate_Lk_by_Ck(data_set, c1, min_support, support_data)
    Lksub1 = L1.copy()
    L = []
    L.append(Lksub1)
    for i in range(2, k+1):
        Ci = create_Ck(Lksub1, i)
        Li = generate_Lk_by_Ck(data_set, Ci, min_support, support_data)
        Lksub1 = Li.copy()
        L.append(Lksub1)
    return L, support_data

In [11]:
L, support_data = generate_L(df, k=17, min_support=0.001)

<h1>Function to find rules from L and support data</h1>

In [12]:
def generate_rules(L, support_data, min_conf):
    rule_list = []
    sub_set_list = []
    for i in range(0, len(L)):
        for freq_set in L[i]:
            for sub_set in sub_set_list:
                if sub_set.issubset(freq_set):
                    conf = support_data[freq_set] / support_data[freq_set - sub_set]
                    rule = (freq_set - sub_set, sub_set, conf)
                    if conf >= min_conf and rule not in rule_list:
                        # print freq_set-sub_set, " => ", sub_set, "conf: ", conf
                        rule_list.append(rule)
            sub_set_list.append(freq_set)
    return rule_list

In [14]:
rules_list = generate_rules(L, support_data, 0.5)

<h1>Creating rules DataFrame with rules and confedence</h1>

In [15]:
rule_df=pd.DataFrame(columns=["rules", "confidence"])
i=0
for item in rules_list:
    rule_df.loc[i]=[f'{item[0]} => {item[1]}', item[2]]
    i+=1
rule_df = rule_df.sort_values("confidence", ascending=False)

<h1>Save the rules DataFrame to a csv file and show the five first rows</h1>

In [16]:
rule_df.to_csv("rules.csv", encoding='utf-8', index=False)
print(len(rules_list))
rule_df.head()

976


,rules,confidence
175,"frozenset({15, 5, 7}) => frozenset({6})",0.902616
748,"frozenset({2, 11, 14, 7}) => frozenset({1})",0.902420
629,"frozenset({10, 11, 12, 14}) => frozenset({1})",0.900000
665,"frozenset({2, 11, 12, 7}) => frozenset({1})",0.896761
953,"frozenset({2, 3, 10, 11, 12}) => frozenset({1})",0.889960
